# Introduction

Today we'll dive deep into a dataset all about LEGO. From the dataset we can ask whole bunch of interesting questions about the history of the LEGO company, their product offering, and which LEGO set ultimately rules them all:

<ul type="square">
<li>What is the most enormous LEGO set ever created and how many parts did it have?</li>

<li>How did the LEGO company start out? In which year were the first LEGO sets released and how many sets did the company sell when it first launched?</li>

<li>Which LEGO theme has the most sets? Is it one of LEGO's own themes like Ninjago or a theme they licensed liked Harry Potter or Marvel Superheroes?</li>

<li>When did the LEGO company really expand its product offering? Can we spot a change in the company strategy based on how many themes and sets did it released year-on-year?</li>

<li>Did LEGO sets grow in size and complexity over time? Do older LEGO 
sets tend to have more or fewer parts than newer sets?</li>
</ul>

**Data Source**

[Rebrickable](https://rebrickable.com/downloads/) has compiled data on all the LEGO pieces in existence. I recommend you use download the .csv files provided in this lesson. 

<img src="./assets/bricks.jpg" alt="bricks photo"> 

## What you'll learn today

How to combine a Notebook with HTML Markup.

Apply Python List slicing techniques to Pandas DataFrames.

How to aggregate data using the ```.agg()``` function.

How to create scatter plots, bar charts, and line charts with two axes in Matplotlib.

Understand database schemas that are organised by primary and foreign keys.

How to merge DataFrames that share a common key

# Import Statements

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Data Exploration

**Challenge**: How many different colours does the LEGO company produce? Read the colors.csv file in the data folder and find the total number of unique colours. Try using the [.nunique() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html?highlight=nunique#pandas.DataFrame.nunique) to accomplish this.

In [ ]:
colors_df = pd.read_csv("./data/colors.csv")
colors_df.head()

In [ ]:
colors_df.tail()

In [ ]:
colors_df["name"].nunique()  # how many unique number of colors we have.

**Challenge**: Find the number of transparent colours where <code>is_trans == 't'</code> versus the number of opaque colours where <code>is_trans == 'f'</code>. See if you can accomplish this in two different ways.

In [ ]:
colors_df["is_trans"].value_counts()
# OR 
# colors_df.groupby("is_trans").count()

### Understanding LEGO Themes vs. LEGO Sets

Walk into a LEGO store, and you will see their products organised by theme. Their themes include Star Wars, Batman, Harry Potter and many more.

<img src="./assets/lego_themes.png" alt="Lego Themes">

A lego <b>set</b> is a particular box of LEGO or product. Therefore, a single theme typically has many different sets.
<img src="./assets/lego_sets.png">

The <code>sets.csv</code> data contains a list of sets over the years and the number of parts that each of these sets contained.

**Challenge**: Read the sets.csv data and take a look at the first and last couple of rows. 

In [ ]:
sets_df = pd.read_csv("./data/sets.csv")

In [ ]:
sets_df.head()

In [ ]:
sets_df.tail()

**Challenge**: In which year were the first LEGO sets released and what were these sets called?

In [ ]:
sets_df.loc[sets_df['year'].idxmin()]  # 1949

**Challenge**: How many different sets did LEGO sell in their first year? How many types of LEGO products were on offer in the year the company started?

In [ ]:
year_id = sets_df["year"].idxmin()  # 9521
sets_df[sets_df["year"] == 1949]

**Challenge**: Find the top 5 LEGO sets with the most number of parts. 

In [ ]:
top_sets = sets_df.sort_values(by="num_parts", ascending=False)
top_sets.head()

**Challenge**: Use <code>.groupby()</code> and <code>.count()</code> to show the number of LEGO sets released year-on-year. How do the number of sets released in 1955 compare to the number of sets released in 2019? 

In [ ]:
sets_df.groupby("year").count()

In [ ]:
sets_by_year = sets_df.groupby("year").count()
sets_by_year["set_num"].head()

**Challenge**: Show the number of LEGO releases on a line chart using Matplotlib. <br>
<br>
Note that the .csv file is from late 2020, so to plot the full calendar years, you will have to exclude some data from your chart. Can you use the slicing techniques covered in Day 21 to avoid plotting the last two years? The same syntax will work on Pandas DataFrames. 

In [ ]:
plt.xlabel("Year")
plt.ylabel("Sets Number")
# excluding last two years, include all rows, include column from index -2 (starts from the end, skips first two from the end)
plt.plot(sets_by_year.index[:-2], sets_by_year["set_num"][:-2])

### Aggregate Data with the Python .agg() Function

Let's work out the number of different themes shipped by year. This means we have to count the number of unique theme_ids per calendar year.
In our case, we want to calculate the number of different themes by calendar year. This means we have to group the data by year and then count the number of unique ```theme_id``` for that year.

## Number of Themes per Calendar Year

We can accomplish this by chaining the ```.groupby()``` and the ```.agg()``` functions together:

In [ ]:
themes_by_year = sets_df.groupby("year").agg({"theme_id": pd.Series.nunique})

In [ ]:
themes_by_year.rename(columns={"theme_id": "nr_themes"}, inplace=True)
themes_by_year.tail()

**Challenge**: Plot the number of themes released by year on a line chart. Only include the full calendar years (i.e., exclude 2020 and 2021). 

In [ ]:
plt.xlabel("Years")
plt.ylabel("Theme Numbers")
plt.plot(themes_by_year.index[:-2], themes_by_year["nr_themes"][:-2])

### Line Charts with Two Separate Axes
Goal of creating separate axes lays about if we have 2 different plots on the diagram and the 2 plots have different scale,
let's say **theme_by_year** range from 0 to 90,  but **sets_by_year** range between 0 and 900 which will lead to a confusion state, for example <br>
<img src="assets/two-plots.png"><br>
Well, that's not very informative!️ The problem is that the "number of themes" and the "number of sets" have very different scales. The theme number ranges between 0 and 90, while the number of sets ranges between 0 and 900. So what can we do?

## Two Separate Axes

We need to be able to configure and plot our data on two separate axes on the same chart. This involves getting hold of an axis object from Matplotlib.

In [ ]:
#ax1 = plt.gca()  # get the axis
#ax2 = plt.twinx(ax1)  # create another axis that shares the same x-axis

We then create another axis object: ax2. The key thing is that by using the ```.twinx()``` method allows ax1 and ax2 to share the same x-axis. When we plot our data on the axes objects we get this:

In [ ]:
ax1 = plt.gca()  # get current axes
ax2 = ax1.twinx()  # Create a twin Axes sharing the x-axis
ax1.plot(sets_by_year['set_num'][:-2], 'g:')
ax2.plot(themes_by_year[:-2], color='r')  # excluding last 2 years 
ax1.legend(['set_num'], loc="upper left")
ax2.legend(['themes_by_year'], loc='lower right')
ax1.set_xlabel('Year')
ax1.set_ylabel('Number of Sets', color='green')
ax2.set_ylabel('Number of Themes', color='red')

### Scatter Plots in Matplotlib

**Challenge**: Has the size and complexity of LEGO sets increased over time based on the number of parts? Plot the average number of parts over time using a Matplotlib scatter plot. See if you can use the [scatter plot documentation](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.pyplot.scatter.html) before I show you the solution. Do you spot a trend in the chart? 

## Challenge


Create a Pandas Series called parts_per_set that has the year as the index and contains the average number of parts per LEGO set in that year. Here's what you're looking to create:

In [ ]:
parts_per_set = sets_df.groupby("year").agg({"num_parts": "mean"})
parts_per_set.head()

In [ ]:
parts_per_set.tail()

In [ ]:
plt.xlabel("Years")
plt.ylabel("Part Numbers")
plt.scatter(parts_per_set.index[:-2], parts_per_set["num_parts"][:-2])

### Number of Sets per LEGO Theme

LEGO has licensed many hit franchises from Harry Potter to Marvel Super Heros to many others. But which theme has the largest number of individual sets? 

In [ ]:
set_theme_count = sets_df["theme_id"].value_counts()
set_theme_count

We have no idea what our themes are actually called! Ok, we can see that the theme with id 158 is the largest theme containing 753 individual sets, but what's that theme called? This is not very helpful. We need to find the names of the themes based on the theme_id from the themes.csv file.

**Challenge** Use what you know about HTML markup and tags to display the database schema: https://i.imgur.com/Sg4lcjx.png


**Schema**<br>
<img src="assets/rebrickable_schema.png">

### Database Schemas, Foreign Keys and Merging DataFrames

The ```themes.csv``` file has the actual theme names. The sets .csv has <code>theme_ids</code> which link to the <code>id</code> column in the themes.csv. 

**Challenge**: Explore the themes.csv. How is it structured? Search for the name 'Star Wars'. How many <code>id</code>s correspond to this name in the themes.csv? Now use these <code>id</code>s and find the corresponding the sets in the sets.csv (Hint: you'll need to look for matches in the <code>theme_id</code> column)

In [ ]:
themes_df = pd.read_csv("./data/themes.csv")

In [ ]:
themes_df.head()

In [ ]:
themes_df.tail()

In [ ]:
themes_df[themes_df["name"] == "Star Wars"]

In [ ]:
sets_df[sets_df["theme_id"] == 209]

### Merging (i.e., Combining) DataFrames based on a Key


In [ ]:
set_theme_count  # let's give theme_id a name!

In [ ]:
# to merge we need to convert the pandas series into a dataframe
set_theme_count = pd.DataFrame({
    "id": set_theme_count.index,
    "set_count": set_theme_count.values
})
set_theme_count

In [ ]:
merged_df = pd.merge(set_theme_count, themes_df, on="id")
merged_df

In [ ]:
plt.bar(merged_df.name[:10], merged_df.set_count[:10])  # not readable

In [ ]:
plt.figure(figsize=(14,8))
plt.xticks(fontsize=14, rotation=45)
plt.yticks(fontsize=14)
plt.ylabel('Nr of Sets', fontsize=14)
plt.xlabel('Theme Name', fontsize=14)
plt.bar(merged_df.name[:10], merged_df.set_count[:10])